In [1]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import sys
import anndata as an
import scanpy as sp
import h5py

sys.path.append('../src')
sys.path.append('../tests')

from lightning_train import DataModule, generate_trainer
from data import *
from model import *
from testing import *

from pytorch_lightning.loggers import WandbLogger
from torchmetrics.functional import *

import pandas as pd
from scipy.sparse import csr_matrix
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from bigcsv.bigcsv import experimental_to_h5ad
from functools import partial
import torchmetrics.functional as f
from torchmetrics import Metric
import torchmetrics 

In [2]:
data = an.read_h5ad('../data/pancreas/pancreas.h5ad')
data

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


AnnData object with n_obs × n_vars = 14693 × 2448
    obs: 'celltype', 'sample', 'n_genes', 'batch', 'n_counts', 'louvain'
    var: 'n_cells-0', 'n_cells-1', 'n_cells-2', 'n_cells-3'
    uns: 'celltype_colors', 'louvain', 'neighbors', 'pca', 'sample_colors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

Now that we know all the labels are the same, we can subset the samples we'd like to train on and train a model

In [5]:
for i in range(4):
    labels = data.obs[data.obs['batch'] == str(i)]
    labels.to_csv(f'../data/pancreas/batch_{i}_labels.csv', index=False)

# Test refgenes to make sure this code is buggy .. 

In [10]:
currgenes = data.var.index
refgenes = data.var.index[10:]

In [11]:
len(currgenes), len(refgenes)

(2448, 2438)

In [12]:
datamodule = DataModule(
    datafiles=['../data/pancreas/pancreas.h5ad'],
    labelfiles=['../data/pancreas/batch_0_labels.csv'],
    class_label='celltype',
    assume_numeric_label=False,
    refgenes=refgenes,
    currgenes=currgenes,
)

datamodule.prepare_data()
datamodule.setup()

model = TabNetLightning(
    input_dim=datamodule.num_features,
    output_dim=datamodule.num_labels,
)

assume_numeric_label=False, using sklearn.preprocessing.LabelEncoder and encoding target variables.
Creating train/val/test DataLoaders...


/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


Done, continuing to training.
Calculating weights
Initializing network
Initializing explain matrix


In [ ]:
wandb_logger = WandbLogger(
    project=f"Pancreas Model",
    name='Local, Default Settings'
)

trainer = pl.Trainer(
    logger=wandb_logger,
    max_epochs=200,
)

trainer.fit(model, datamodule=datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name    | Type   | Params
-----------------------------------
0 | network | TabNet | 166 K 
-----------------------------------
166 K     Trainable params
0         Non-trainable params
166 K     Total params
0.667     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Median f1 score is 0.0 for epoch=0
Validation F1-scores are [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/julian/Documents/Projects/SIMS/notebooks/../src/model.py:196: RuntimeWarning: invalid value encountered in true_divide
  precision = tp / (tp + fp)
/Users/julian/Documents/Projects/SIMS/notebooks/../src/model.py:197: RuntimeWarning: invalid value encountered in true_divide
  recall = tp / (tp + fn)
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (tr

...

Median f1 score is 0.0 for epoch=31
Validation F1-scores are [0.64835165 0.         0.48924731 0.5221843  0.         0.49422633
 0.14285714 0.         0.         0.         0.         0.
 0.         0.        ]
Median f1 score is 0.0 for epoch=31


Validating: 0it [00:00, ?it/s]

Median f1 score is 0.0 for epoch=32
Validation F1-scores are [0.69375    0.         0.46654611 0.50622407 0.         0.56455142
 0.11320755 0.         0.         0.         0.         0.
 0.         0.        ]
Median f1 score is 0.0 for epoch=32


Validating: 0it [00:00, ?it/s]

Median f1 score is 0.0 for epoch=33
Validation F1-scores are [0.64666667 0.         0.5237366  0.44359465 0.         0.52193995
 0.17857143 0.         0.         0.         0.         0.
 0.         0.        ]
Median f1 score is 0.0 for epoch=33


In [77]:
trainer.validate(model, datamodule=datamodule)

Validating: 0it [00:00, ?it/s]

Median f1 score is 0.8883735312306741 for epoch=165
Validation F1-scores are [0.93638677 0.94017094 0.96624473 0.96119403 0.89795918 0.95852535
 0.93069307 0.         0.84210526 0.         0.         0.87878788
 0.         0.        ]
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'val_accuracy': 0.9416569471359253,
 'val_precision': 0.9416569471359253,
 'val_recall': 0.9416569471359253}
--------------------------------------------------------------------------------


[{'val_accuracy': 0.9416569471359253,
  'val_precision': 0.9416569471359253,
  'val_recall': 0.9416569471359253}]

In [35]:
t1 = data.obs[data.obs['batch'] == '0']
t2 = data.obs[data.obs['batch'] == '1']

In [44]:
t1['celltype'].unique().categories

Index(['MHC class II', 'PSC', 'acinar', 'activated_stellate', 'alpha', 'beta',
       'co-expression', 'delta', 'dropped', 'ductal', 'endothelial', 'epsilon',
       'gamma', 'macrophage', 'mast', 'mesenchymal', 'mesenchyme',
       'not applicable', 'quiescent_stellate', 'schwann', 't_cell',
       'unclassified', 'unclassified endocrine', 'unclear'],
      dtype='object')

In [45]:
t2['celltype'].unique().categories == t1['celltype'].unique().categories

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])